<a href="https://colab.research.google.com/github/SilverFoxMedia-Tesis/IA_02/blob/main/RED_NEURONAL_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import keras

# Cargar el dataset para revisar las primeras filas y obtener una descripción general de los datos
file_path = '/train_last_modified_v3.csv'
data = pd.read_csv(file_path)

# Mostrar las primeras filas y la información del dataset
print(data.head())
print(data.info())
print(data.describe())



In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Preprocesamiento de las características
scaler = StandardScaler()
X = scaler.fit_transform(data[['TMB', 'desayuno_carbs', 'desayuno_grasas', 'almuerzo_carbs', 'almuerzo_grasas', 'cena_carbs', 'cena_grasas']])

# Preparar etiquetas
tokenizer = Tokenizer()
# Asumiendo que cada columna de comidas es una lista convertida a cadena. Ajustar según los datos.
data['Desayuno'] = data['Desayuno'].apply(eval)  # Convierte de string de lista a lista real, si es necesario
tokenizer.fit_on_texts(data['Desayuno'].explode().astype(str))
vocab_size = len(tokenizer.word_index) + 1

Y_desayuno = tokenizer.texts_to_sequences(data['Desayuno'].astype(str))
Y_desayuno_padded = pad_sequences(Y_desayuno, padding='post', maxlen=10)  # Ajusta 'maxlen' según la longitud necesaria

# Dividir los datos en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_desayuno_padded, test_size=0.2, random_state=42)

# Imprimir tamaños para verificar
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (23517, 7)
Y_train shape: (23517, 10)
X_test shape: (5880, 7)
Y_test shape: (5880, 10)


In [ ]:
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.16.1
Keras version: 3.2.1


In [ ]:
pip install tensorflow==2.16.1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Reshape, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Cargar el dataset
file_path = '/train_last_modified_v3.csv'
data = pd.read_csv(file_path)

# Preprocesamiento de las características de entrada
scaler = StandardScaler()
X = scaler.fit_transform(data[['Carbohidratos_G', 'Proteinas_G', 'Grasas_G', 'TMB']])

# Preparar las etiquetas para desayuno, almuerzo, cena y rutina
tokenizer = Tokenizer()
data['Desayuno'] = data['Desayuno'].apply(eval)
data['Almuerzo'] = data['Almuerzo'].apply(eval)
data['Cena'] = data['Cena'].apply(eval)
data['Rutina'] = data['Rutina'].apply(eval)

# Tokenizar y secuenciar las columnas de comida
columns = ['Desayuno', 'Almuerzo', 'Cena', 'Rutina']
for column in columns:
    tokenizer.fit_on_texts(data[column].explode().astype(str))

vocab_size = len(tokenizer.word_index) + 1

# Concatenar las secuencias tokenizadas de desayuno, almuerzo, cena y rutina
Y = np.zeros((len(data), 4, 10))  # Cambiar 10 por la longitud de secuencia deseada
for i, column in enumerate(columns):
    sequences = tokenizer.texts_to_sequences(data[column].astype(str))
    padded_sequences = pad_sequences(sequences, padding='post', maxlen=10)
    Y[:, i, :] = padded_sequences

In [ ]:
# Asumiendo que 'Y' tiene la forma (num_samples, 4, 10), donde cada entrada es un índice de clase
Y_cat = np.zeros((Y.shape[0], 4, 10, vocab_size))  # Convertir a formato categórico

for i in range(Y.shape[0]):
    for j in range(4):  # Por cada uno de los 4 segmentos (Desayuno, Almuerzo, Cena, Rutina)
        for k in range(10):  # Por cada posición en la secuencia de 10
            Y_cat[i, j, k, :] = to_categorical(Y[i, j, k], num_classes=vocab_size)

# Ahora aplana correctamente la última dimensión
Y_train = np.sum(Y_cat, axis=2)  # Suma a lo largo de la secuencia para colapsar en una única distribución de probabilidad

# Dividir los datos en entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_train, test_size=0.2, random_state=42)

# Construir el modelo
model = Sequential([
    Input(shape=(4,)),  # 4 características de entrada
    Dense(64, activation='relu'),
    Dense(4 * vocab_size, activation='relu'),  # Ajustar tamaño según vocab_size total
    Reshape((4, vocab_size)),  # Asegurarse de que esta forma sea correcta
    LSTM(100, return_sequences=True),
    Dropout(0.5),
    Dense(vocab_size, activation='sigmoid')  # Cambio clave aquí
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Entrenar el modelo
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

# Guardar y manejar la descarga del modelo
model.save('fitLifeBuddy_IA_v4.keras')


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                     │ (None, 64)                  │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 324)                 │          21,060 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_9 (Reshape)                  │ (None, 4, 81)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ (None, 4, 100)              │          72,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 4, 100)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 4, 81)               │           8,181 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 102,361 (399.85 KB)

 Trainable params: 102,361 (399.85 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9836 - loss: -1.3750 - val_accuracy: 1.0000 - val_loss: -5.1248
Epoch 2/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 1.0000 - loss: -6.2588 - val_accuracy: 1.0000 - val_loss: -9.6590
Epoch 3/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 1.0000 - loss: -10.7793 - val_accuracy: 1.0000 - val_loss: -14.1509
Epoch 4/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 1.0000 - loss: -15.2816 - val_accuracy: 1.0000 - val_loss: -18.6300
Epoch 5/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 1.0000 - loss: -19.7505 - val_accuracy: 1.0000 - val_loss: -23.1037
Epoch 6/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 1.0000 - loss: -24.2146 - val_accuracy: 1.0000 - val_loss: -27.5739
Epoch 7/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 1.0000 - loss: -28.6952 - val_accuracy: 1.0000 - val_loss: -32.0447
Epoch 8/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 1.0000 - l

In [ ]:
# Descargar el archivo al sistema local
files.download('fitLifeBuddy_IA_v4.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np

# Datos de entrada nuevos
nuevos_datos = np.array([[174.90, 69.96, 46.64, 1749.00]])  # Nota que los datos están en un array 2D

# Escalar los datos usando el mismo escalador que se usó para el entrenamiento
# Asegúrate de que el objeto 'scaler' ha sido ajustado con los datos de entrenamiento y está disponible aquí
nuevos_datos_escalados = scaler.transform(nuevos_datos)

# Hacer la predicción
predicciones = model.predict(nuevos_datos_escalados)
print("Predicciones crudas:", predicciones)

# Decodificar las predicciones si es necesario
# Si tienes clases categóricas y las predicciones son probabilidades, puedes obtener las clases predichas como sigue:
predicciones_clases = np.argmax(predicciones, axis=-1)
print("Clases predichas:", predicciones_clases)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicciones crudas: [[[1.00000000e+00 2.28729680e-01 2.19436705e-01 1.47879466e-01
   1.86184317e-01 3.84441257e-01 3.03784847e-01 1.63877964e-01
   1.35698542e-01 1.80854738e-01 1.75731748e-01 1.12598367e-01
   7.55942389e-02 8.10212120e-02 5.86023517e-02 7.22034648e-02
   5.49563579e-02 3.89199220e-02 4.51321192e-02 3.80308665e-02
   4.21710201e-02 4.46306430e-02 4.36855704e-02 3.27328518e-02
   3.63146700e-02 4.10664417e-02 3.78908627e-02 3.65724303e-02
   4.01734672e-02 3.69698964e-02 3.97150591e-02 4.12609689e-02
   3.87508683e-02 3.37607153e-02 3.51826847e-02 3.56404781e-02
   3.32570672e-02 3.49258147e-02 3.46491858e-02 3.85389738e-02
   3.70562859e-02 4.05040495e-02 3.71341743e-02 2.99489573e-02
   3.13990973e-02 3.32873799e-02 3.43628787e-02 3.45238596e-02
   3.78200300e-02 3.79426442e-02 3.18674371e-02 3.60422358e-02
   3.42724286e-02 3.94076779e-02 3.78837734e-02 3.11695375e-02
   3.66222337e-02 3.10142543e-02 3.71174216e-02 3.72030586e

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
def obtener_etiquetas_activas(predicciones, umbral=0.2):
    return (predicciones > umbral).astype(int)

def predicciones_a_ids(predicciones, indice_a_id, umbral=0.2):
    predicciones_binarias = obtener_etiquetas_activas(predicciones, umbral)
    ids_activos = []
    for segmento_predicciones in predicciones_binarias:
        ids = [indice_a_id[i] for i, activo in enumerate(segmento_predicciones) if activo == 1]
        ids_activos.append(ids)
    return ids_activos

# Ejemplo de uso tras entrenamiento
predicciones = model.predict(X_test)
ids_predichos = predicciones_a_ids(predicciones[0], indice_a_id)  # Asume indice_a_id es tu mapeo de índice a ID
print("IDs predichos:", ids_predichos)


184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


NameError: name 'indice_a_id' is not defined